# Legal Contract Ticket Decision Engine

## Overview
This assignment trains you to think like a **real system**, not like a lawyer and not like a chatbot.

Your system does **one job only**:
> Decide the **next action** for a user request related to **legal contract issues**.

The system **does NOT**:
- Give legal advice
- Explain contract clauses
- Suggest what the user should do

It only decides **what happens next**.

---

## Allowed Actions (Strict)
Your output **must be one of these five actions only**:

```
CREATE_TICKET
ASK_FOR_MORE_INFO
REJECT_REQUEST
FLAG_OUT_OF_SCOPE
ESCALATE_TO_HUMAN
```

Any other action = ❌ **Fail**

---

## Action Definitions

### 1. CREATE_TICKET
Use this action **only if all are true**:
- The issue is legal
- The issue is related to a contract
- A real dispute is described
- Information is enough to open a ticket

To create a ticket, the message must clearly indicate:
- A written agreement or contract
- At least two parties
- A specific dispute or legal event
If any of these are missing, do NOT create a ticket.

Example:
> "The tenant left before the end date and refused to pay the penalty."

---

### 2. ASK_FOR_MORE_INFO
Use this action if:
- The issue is clearly contract-related
- But important details are missing

Example:
> "There is a problem with the agreement."

---

### 3. REJECT_REQUEST
Use this action if the user:
- Asks for legal advice
- Asks to explain a contract clause
- Asks what they should do

Example:
> "Is this penalty clause fair?"

---

### 4. FLAG_OUT_OF_SCOPE
Use this action if the request is:
- Not legal
- Legal but **not related to contracts**
- Personal or general complaints

Example:
> "Someone threatened me yesterday."

---

### 5. ESCALATE_TO_HUMAN
Use this action if:
- The issue is contract-related
- The situation is very sensitive or risky
- The system should not decide alone

Example:
> "The partner used a loophole in the agreement to take all profits."

---

## Input Format
You receive **only one input**:

```json
{
  "user_message": "string"
}
```

No extra fields. You must understand everything from the text.

---

## Output Format (Flexible)


You are **NOT required** to use a specific output structure or JSON format.

You may present your output in **any format you prefer**, such as:
- JSON
- Table
- Bullet points

### What matters is **NOT the shape**, but the **decision**.

Your output must clearly show:
1. **The chosen action** (one of the allowed actions)
2. **Your confidence level** (any reasonable scale is acceptable)
3. **A short explanation** of why this action was selected

### Example (one possible format)

Action: ASK_FOR_MORE_INFO  
Confidence: Medium  
Reasoning: The message suggests a contract-related dispute, but key details are missing.
#### Or
```json
{
  "action": "ONE_OF_THE_FIVE_ACTIONS",
  "confidence": 0.0,
  "reasoning": "Short, neutral explanation"
}
```
### Important Notes
- Do **not** give legal advice
- Do **not** explain laws or contract clauses
- Focus only on deciding the **next system action**
- Keep reasoning short and neutral



You are free to design the output format in the way that best helps you explain your thinking.

----


# Notes:

---
## 1) What Does a Good Answer Look Like?

A good answer is **not** about choosing the “perfect” action.

A good answer:
- Respects the system boundaries
- Chooses one of the allowed actions
- Explains the decision clearly and logically

If your reasoning is strong and consistent,  
you may receive full credit even if another action could also be reasonable.

---
## 2) Real-World Note

In real companies, systems like this fail not because they are “not smart enough”,
but because they act when they should not.

This assignment trains you to build systems that:
- Know their limits
- Make safe decisions
- Protect users and the company

---
## 3) How This Assignment Is Structured

This assignment is divided into **three parts**, each one building on the previous one.

Do **not** treat the parts as separate tasks.  
They represent **progressive levels of system thinking**:

- First, you learn where the system **must stop**
- Then, you decide when the system **can act**
- Finally, you handle **uncertainty and risk**

Read each part carefully before starting.  
The same rules and allowed actions apply to **all parts**.



In [ ]:
from google.colab import drive
import torch
drive.mount('/content/drive')

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_path =  "Your Path"

tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    local_files_only=True
)

from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
      quantization_config=bnb_config,
  torch_dtype=torch.float16,
    local_files_only=True
)

print("✅ Model loaded locally from Drive")


## Quick Check Task — Model Run

### Goal
Confirm that the model is running and responding correctly.


### Instructions
1. Ask the model one simple question.
2. Make sure it returns a clear answer.
3. Copy the question and the response.



---

# Part 1 — Scope Control (Beginner)

### Goal
Learn when the system **should not act**.

### Task
For each message, choose the correct action.

| User Message | Expected Action |
|-------------|----------------|
| "I want to know my rights" | REJECT_REQUEST |
| "There is a dispute with a company" | ASK_FOR_MORE_INFO |
| "I was threatened" | FLAG_OUT_OF_SCOPE |

### Deliverable
- Your decision rules (text)
- 10 examples written by you

---



# Part 2 — Information Judgment (Intermediate)

### Goal
Decide if the information is **enough** to create a ticket.

### Task
- Choose an action
- Explain your reasoning briefly

Example:

```json
{
  "action": "ASK_FOR_MORE_INFO",
  "confidence": 0.62,
  "reasoning": "A contract dispute is implied, but contract type and parties are missing."
}
```

### Deliverable
- A checklist for CREATE_TICKET
- 10 difficult examples

---



# Part 3 — Ambiguity & Risk (Advanced)

### Goal
Handle unclear and risky situations.

### Rules
- The word "contract" may NOT appear in the message

Examples:

> "The company ended the cooperation suddenly despite our written agreement"

Expected:
- ASK_FOR_MORE_INFO **or** ESCALATE_TO_HUMAN

> "Can this clause force me to pay a penalty?"

Expected:
- REJECT_REQUEST

### Deliverable
- 10 ambiguous messages
- Chosen action for each
- Why other actions were NOT chosen

---



# Part 4 (Optional) — Decision Maker API using FastAPI



## Why This Task Matters

In real companies, LLMs are **not used as notebooks or demos**.
They are deployed as **services** that other systems depend on.

Product teams, backend services, and automation pipelines
cannot "talk" to a chatbot.
They need **stable, predictable decisions** exposed through APIs.

This task simulates how LLMs are actually used in production:

* As a decision layer
* Behind an HTTP interface
* With strict contracts and validation

This is the same pattern used in:

* Ticketing systems
* Internal automation tools
* Agent-based platforms
* AI-powered workflows in companies

---

## Task Type

**Optional (Advanced / Production-Oriented)**

This task is for students who want to:

* Think like production engineers
* Understand real-world AI system design
* Go beyond notebooks into deployable systems

---

## Task Goal

Convert your **LLM Decision Maker** into a real **API service**.

The LLM will no longer generate text for humans.
It will:

* Receive a request
* Decide what action to take
* Return a strict machine-readable response

---

## What You Will Build

You will build a **FastAPI service** that exposes a single endpoint
used as a **Decision Layer**.

This API represents how companies integrate LLMs into systems.

---

## API Specification

### Endpoint

```
POST /decide
```

### Input (JSON)

```json
{
  "user_request": "I want to report a payment issue",
  "user_id": "12345"
}
```

### Output from LLM (JSON)

```json
{
  "action": "CREATE_TICKET",
  "priority": "HIGH"
}
```

No explanations.
No free text.
Only valid schema output.

---

## System Rules (Mandatory)

Your LLM must follow these rules:

* Output must match the response schema exactly
* No additional text is allowed
* Same input must produce the same output
* No guessing or assumptions
* Any schema violation is considered a failure

---

## Decision Logic Requirements

Inside the LLM decision module:

* Use closed output space
* Use deterministic decoding
* Enforce strict rules
* Validate output before returning it

The API must never trust raw LLM output.

---

## Error Handling

* If the LLM output is invalid → retry once
* If it fails again → return HTTP 400
* Do not interpret or fix invalid outputs

Fail fast, as in real systems.

---

## What NOT To Do

* Do not build a chat interface
* Do not allow free-text responses
* Do not enable sampling or temperature
* Do not bypass schema validation

---

## Deliverables

Students who choose this task must submit:

1. FastAPI project folder
2. Working `/decide` endpoint
3. README.md explaining:

   * System purpose
   * API contract
   * Decision rules

---

## Final Note

This task reflects how AI engineers work in real companies.

You are not training a model.
You are **building a system** that other systems rely on.